In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import TimeSeriesSplit

import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 30)

#sns.set_style("whitegrid")
#plt.style.use('bmh')
plt.style.use('seaborn-whitegrid')

# this allows plots to appear directly in the notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
train_df = pd.read_csv('data/train.csv')
train_df['data_set'] = 'train'
test_df = pd.read_csv('data/test.csv')
test_df['data_set'] = 'test'
# combine train and test data into one df
test_df['registered'] = 0
test_df['casual'] = 0
test_df['count'] = 0

all_df = pd.concat([train_df, test_df])
# parse datetime colum & add new time related columns
dt = pd.DatetimeIndex(all_df['datetime'])
all_df.set_index(dt, inplace=True)

# logarithmic transformation of dependent cols
# (adding 1 first so that 0 values don't become -inf)
for col in ['casual', 'registered', 'count']:
    all_df[f'{col}_log'] = np.log(all_df[col] + 1)

all_df['date'] = dt.date # yyyymmdd
all_df['day'] = dt.day # dd
all_df['month'] = dt.month # mm
all_df['year'] = dt.year # yyyy
all_df['hour'] = dt.hour # hh
all_df['dow'] = dt.dayofweek #曜日 Mon:0 Tue:1 Wed:2 Thu:3 Fri:4 Sat:5 Sun:6
all_df['woy'] = dt.isocalendar().week #その日の週が年間で見ると何番目の週かを表す数字 [dt.weekofyear]は deprecated
all_df.head(3)

# feature engineer a new column whether its a peak hour or not
data = all_df[all_df['data_set'] == 'train'].copy().groupby(["hour","workingday"])["count"].mean().reset_index()
data = data.rename(columns={'count':'mean_of_count_on_workingday'})
all_df = pd.merge(all_df,data,on=['hour','workingday'])
all_df = all_df.sort_values('datetime')
dt = pd.DatetimeIndex(all_df['datetime'])
all_df.set_index(dt, inplace=True)
subset = all_df.query('mean_of_count_on_workingday < 100 & mean_of_count_on_workingday >=0')
all_df.loc[subset.index, 'peak'] = 0
subset = all_df.query('mean_of_count_on_workingday < 200 & mean_of_count_on_workingday >=100')
all_df.loc[subset.index, 'peak'] = 1
subset = all_df.query('mean_of_count_on_workingday < 300 & mean_of_count_on_workingday >=200')
all_df.loc[subset.index, 'peak'] = 2
subset = all_df.query('mean_of_count_on_workingday < 400 & mean_of_count_on_workingday >=300')
all_df.loc[subset.index, 'peak'] = 3
subset = all_df.query('mean_of_count_on_workingday < 500 & mean_of_count_on_workingday >=400')
all_df.loc[subset.index, 'peak'] = 4
subset = all_df.query('mean_of_count_on_workingday >= 500')
all_df.loc[subset.index, 'peak'] = 5



# add a count_season column using join
by_season = all_df[all_df['data_set'] == 'train'].copy().groupby(['season'])[['count']].agg(sum)
by_season.columns = ['count_season']
all_df = all_df.join(by_season, on='season')


#ここの修正の仕方は、間違っているので要修正！
# sandy
#all_df['holiday'] = all_df[['month', 'day', 'holiday', 'year']]\
#    .apply(lambda df: 1 if (df['year'] == 2012 and df['month'] == 10 and df['day'] == 30) else 0, axis = 1)
# 修正後↓
all_df['holiday'] = all_df[['month', 'day', 'holiday', 'year']]\
    .apply(lambda df: 1 if (df['year'] == 2012 and df['month'] == 10 and df['day'] == 30) else df['holiday'], axis = 1)


# christmas and others
all_df['holiday'] = all_df[['month', 'day', 'holiday']]\
    .apply(lambda df: 1 if (df['month'] == 12 and df['day'] in [24, 26, 31]) else df['holiday'], axis = 1)
all_df['workingday'] = all_df[['month', 'day', 'workingday']]\
    .apply(lambda df: 0 if df['month'] == 12 and df['day'] in [24, 31] else df['workingday'], axis = 1)
# これは流石に気づかない気がする。。。気づけない気がする。。。
def get_day(day_start):
    day_end = day_start + pd.offsets.DateOffset(hours=23)
    return pd.date_range(day_start, day_end, freq="H")

# tax day
all_df.loc[get_day(datetime(2011, 4, 15)), "workingday"] = 1
all_df.loc[get_day(datetime(2012, 4, 16)), "workingday"] = 1

# thanksgiving friday
all_df.loc[get_day(datetime(2011, 11, 25)), "workingday"] = 0
all_df.loc[get_day(datetime(2012, 11, 23)), "workingday"] = 0

# tax day
all_df.loc[get_day(datetime(2011, 4, 15)), "holiday"] = 0
all_df.loc[get_day(datetime(2012, 4, 16)), "holiday"] = 0

# thanksgiving friday
all_df.loc[get_day(datetime(2011, 11, 25)), "holiday"] = 1
all_df.loc[get_day(datetime(2012, 11, 23)), "holiday"] = 1

#storms
all_df.loc[get_day(datetime(2012, 5, 21)), "holiday"] = 1

#tornado
all_df.loc[get_day(datetime(2012, 6, 1)), "holiday"] = 1
# from histogram
all_df['ideal'] = all_df[['temp', 'windspeed']]\
    .apply(lambda df: 1 if (df['temp'] > 27 and df['windspeed'] < 30) else 0, axis = 1)
    
all_df['sticky'] = all_df[['humidity', 'workingday']]\
    .apply(lambda df: 1 if (df['humidity'] >= 60) else 0, axis = 1)

# temperature
all_df.loc[all_df['temp'] < 10,'temp_cat'] = 1
all_df.loc[(all_df['temp'] >= 10) & (all_df['temp'] < 15),'temp_cat'] = 2
all_df.loc[(all_df['temp'] >= 15) & (all_df['temp'] < 20),'temp_cat'] = 3
all_df.loc[(all_df['temp'] >= 20) & (all_df['temp'] < 25),'temp_cat'] = 4
all_df.loc[(all_df['temp'] >= 25) & (all_df['temp'] < 30),'temp_cat'] = 5
all_df.loc[(all_df['temp'] >= 30) & (all_df['temp'] < 35),'temp_cat'] = 6
all_df.loc[(all_df['temp'] >= 35),'temp_cat'] = 7

# humidity category
all_df.loc[all_df['humidity'] < 10,'humidity_cat'] = 0
all_df.loc[(all_df['humidity'] >= 10) & (all_df['humidity'] < 20),'humidity_cat'] = 1
all_df.loc[(all_df['humidity'] >= 20) & (all_df['humidity'] < 30),'humidity_cat'] = 2
all_df.loc[(all_df['humidity'] >= 30) & (all_df['humidity'] < 40),'humidity_cat'] = 3
all_df.loc[(all_df['humidity'] >= 40) & (all_df['humidity'] < 50),'humidity_cat'] = 4
all_df.loc[(all_df['humidity'] >= 50) & (all_df['humidity'] < 60),'humidity_cat'] = 5
all_df.loc[(all_df['humidity'] >= 60) & (all_df['humidity'] < 70),'humidity_cat'] = 6
all_df.loc[(all_df['humidity'] >= 70) & (all_df['humidity'] < 80),'humidity_cat'] = 7
all_df.loc[(all_df['humidity'] >= 80) & (all_df['humidity'] < 90),'humidity_cat'] = 8
all_df.loc[(all_df['humidity'] >= 90),'humidity_cat'] = 9

# windspeed
all_df.loc[all_df['windspeed'] < 5,'wind_cat'] = 0
all_df.loc[(all_df['windspeed'] >= 5) & (all_df['windspeed'] < 10),'wind_cat'] = 1
all_df.loc[(all_df['windspeed'] >= 10) & (all_df['windspeed'] < 15),'wind_cat'] = 2
all_df.loc[(all_df['windspeed'] >= 15) & (all_df['windspeed'] < 20),'wind_cat'] = 3
all_df.loc[(all_df['windspeed'] >= 20) & (all_df['windspeed'] < 25),'wind_cat'] = 4
all_df.loc[(all_df['windspeed'] >= 25) & (all_df['windspeed'] < 30),'wind_cat'] = 5
all_df.loc[(all_df['windspeed'] >= 30) & (all_df['windspeed'] < 35),'wind_cat'] = 6
all_df.loc[(all_df['windspeed'] >= 35) & (all_df['windspeed'] < 40),'wind_cat'] = 7
all_df.loc[(all_df['windspeed'] >= 40) & (all_df['windspeed'] < 45),'wind_cat'] = 8
all_df.loc[(all_df['windspeed'] >= 45),'wind_cat'] = 9


In [3]:
all_df.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,data_set,casual_log,registered_log,count_log,date,day,month,year,hour,dow,woy,mean_of_count_on_workingday,peak,count_season,ideal,sticky,temp_cat,humidity_cat,wind_cat
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,train,1.386294,2.639057,2.833213,2011-01-01,1,1,2011,0,5,52,94.489655,0.0,312498,0,1,1.0,8.0,0.0
2011-01-01 01:00:00,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,train,2.197225,3.496508,3.713572,2011-01-01,1,1,2011,1,5,52,71.910345,0.0,312498,0,1,1.0,8.0,0.0
2011-01-01 02:00:00,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,train,1.791759,3.332205,3.496508,2011-01-01,1,1,2011,2,5,52,53.748252,0.0,312498,0,1,1.0,8.0,0.0


In [4]:
all_df.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count',
       'data_set', 'casual_log', 'registered_log', 'count_log', 'date', 'day',
       'month', 'year', 'hour', 'dow', 'woy', 'mean_of_count_on_workingday',
       'peak', 'count_season', 'ideal', 'sticky', 'temp_cat', 'humidity_cat',
       'wind_cat'],
      dtype='object')

In [5]:
req_cols = ['season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 
        'day','month', 'year', 'hour', 'dow', 'woy', 'peak', 'ideal',
       'sticky', 'temp_cat', 'humidity_cat', 'wind_cat'
       #'datetime', 'casual', 'registered', 'count','data_set', 'casual_log', 'registered_log', 'count_log', 'date','count_season', 
       ]


In [6]:

""" X_train, y_train_r, y_train_c = prep_train_data(train_df, req_cols)

selector = SelectFromModel(RandomForestRegressor(n_estimators = 100,random_state = 42),threshold = 0.3)

selector.fit(X_train,y_train_r)
X_train_f_select_r = selector.transform(X_train) """

' X_train, y_train_r, y_train_c = prep_train_data(train_df, req_cols)\n\nselector = SelectFromModel(RandomForestRegressor(n_estimators = 100,random_state = 42),threshold = 0.3)\n\nselector.fit(X_train,y_train_r)\nX_train_f_select_r = selector.transform(X_train) '

In [7]:
""" feature_idx = selector.get_support()
X_train.columns[feature_idx] """

' feature_idx = selector.get_support()\nX_train.columns[feature_idx] '

In [16]:
# instead of randomly splitting our training data 
# for cross validation, let's construct a framework that's more
# in line with how the data is divvied up for this competition
# (given first 19 days of each month, what is demand for remaining days)
# so, let's split our training data into 2 time contiguous datasets
# for fitting and validating our model (days 1-14 vs. days 15-19).

# also, since submissions are evaluated based on the
# root mean squared logarithmic error (RMSLE), let's replicate
# that computation as we test and tune our model.

train_df = all_df[all_df['data_set'] == 'train']
test_df = all_df[all_df['data_set'] == 'test']

def get_rmsle(y_pred, y_actual):
    diff = np.log(y_pred + 1) - np.log(y_actual + 1)
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)

def custom_train_valid_split(data, cutoff_day=15):
    train = data[data['day'] <= cutoff_day]
    valid = data[data['day'] > cutoff_day]

    return train, valid

def prep_train_data(data, input_cols):
    X = data[input_cols].values
    y_r = data['registered_log'].values
    y_c = data['casual_log'].values

    return X, y_r, y_c

# predict on validation set & transform output back from log scale
def feature_select_on_randomforest(input_cols):
    
    #train, valid = custom_train_valid_split(train_df)
    y_pred_comb_l = []
    y_actual_comb_l = []

    for year_val in [2012]:
        for month_val in range(1,13):

            print(f'Now,{year_val} {month_val} training and validating...')
            # split data for kaggle data rules
            train_tmp = train_df.query('year <= @year_val and month <= @month_val')
            #valid_tmp = valid.query('year == @year_val and month == @month_val')
            
            # split data for feature selection and training, validating
            #X_train, y_train_r, y_train_c = prep_train_data(train_tmp, req_cols)
            #X_valid, y_valid_r, y_valid_c = prep_train_data(valid_tmp, req_cols)

            # feature selection and training and validating
            X, y_r, y_c = prep_train_data(train_tmp, input_cols)
            tscv = TimeSeriesSplit()

            for trn_index, val_index in tscv.split(X):
                #print(f"TRAIN:{trn_index}")
                #print(f"VALIDATE:{val_index}")
                X_train, X_valid = X[trn_index], X[val_index]
                y_train_r, y_valid_r = y_r[trn_index],y_r[val_index]
                y_train_c, y_valid_c = y_c[trn_index],y_c[val_index]


                selector = SelectFromModel(RandomForestRegressor(n_estimators = 100,random_state = 42),threshold = 'median')

                selector.fit(X_train,y_train_r)
                X_train_fs_r = selector.transform(X_train)
                print(f'registered: {train_tmp[input_cols].columns[selector.get_support()].tolist()}')
                
                selector.fit(X_train,y_train_c)
                X_train_fs_c = selector.transform(X_train)
                print(f'casual: {train_tmp[input_cols].columns[selector.get_support()].tolist()}')

    


# predict on test set & transform output back from log scale
""" def predict_on_test_set(model, input_cols):
    
    y_pred_comb_l = []
    for year_val in [2011,2012]:
        for month_val in range(1,13):
            
            # prepare training set
            print(f'Now,{year_val} {month_val} testing...')
            train_df_tmp = train_df.query('year <= @year_val and month <= @month_val')
            test_df_tmp = test_df.query('year == @year_val and month == @month_val')

            X_train, y_train_r, y_train_c = prep_train_data(train_df_tmp, req_cols)

            # prepare testing set
            X_test = test_df_tmp[req_cols]

            # feature selection and testing
            selector = SelectFromModel(RandomForestRegressor(n_estimators = 100,random_state = 42),threshold = 'median')

            selector.fit(X_train,y_train_r)
            X_train_fs_r = selector.transform(X_train)
            print(X_train.columns[selector.get_support()])
            model_r = model.fit(X_train_fs_r, y_train_r)
            X_test_fs_r = selector.transform(X_test)
            y_pred_r = np.exp(model_r.predict(X_test_fs_r)) - 1

            selector.fit(X_train,y_train_c)
            X_train_fs_c = selector.transform(X_train)
            print(X_train.columns[selector.get_support()])
            model_c = model.fit(X_train_fs_c, y_train_c)
            X_test_fs_c = selector.transform(X_test)
            y_pred_c = np.exp(model_c.predict(X_test_fs_c)) - 1
            
            # add casual & registered predictions together
            y_pred_comb = np.round(y_pred_r + y_pred_c)
            y_pred_comb[y_pred_comb < 0] = 0
            y_pred_comb_l.extend(y_pred_comb)

    
    return np.array(y_pred_comb_l) """

" def predict_on_test_set(model, input_cols):\n    \n    y_pred_comb_l = []\n    for year_val in [2011,2012]:\n        for month_val in range(1,13):\n            \n            # prepare training set\n            print(f'Now,{year_val} {month_val} testing...')\n            train_df_tmp = train_df.query('year <= @year_val and month <= @month_val')\n            test_df_tmp = test_df.query('year == @year_val and month == @month_val')\n\n            X_train, y_train_r, y_train_c = prep_train_data(train_df_tmp, req_cols)\n\n            # prepare testing set\n            X_test = test_df_tmp[req_cols]\n\n            # feature selection and testing\n            selector = SelectFromModel(RandomForestRegressor(n_estimators = 100,random_state = 42),threshold = 'median')\n\n            selector.fit(X_train,y_train_r)\n            X_train_fs_r = selector.transform(X_train)\n            print(X_train.columns[selector.get_support()])\n            model_r = model.fit(X_train_fs_r, y_train_r)\n       

In [17]:

feature_select_on_randomforest(req_cols)

Now,2012 1 training and validating...
registered: ['workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'day', 'hour', 'dow', 'woy', 'peak']
casual: ['temp', 'atemp', 'humidity', 'windspeed', 'day', 'hour', 'dow', 'peak', 'temp_cat', 'humidity_cat']
registered: ['workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'day', 'hour', 'dow', 'peak', 'wind_cat']
casual: ['temp', 'atemp', 'humidity', 'windspeed', 'day', 'hour', 'dow', 'peak', 'temp_cat', 'wind_cat']
registered: ['workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'day', 'hour', 'dow', 'peak', 'wind_cat']
casual: ['workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'day', 'hour', 'dow', 'peak', 'temp_cat']
registered: ['workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'day', 'year', 'hour', 'dow', 'peak']
casual: ['workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'day', 'hour', 'dow', 'peak', 'temp_cat']
registered: ['workingday', 'weather', 'temp', 'atemp', 'humidity', 'day', 'year', 'hour', 'dow', 'p